# Pose Bowl: Object Detection


<a id="import-libraries"></a>

## 📚 | Import Libraries 

In [5]:
import sys
sys.path.append('/Users/jsh/code/drivendata/spacecrafts/')

In [6]:
from pathlib import Path
import numpy as np
import pandas as pd

import cv2
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
import torchvision
from torchvision.io import read_image
from torchvision import tv_tensors
from torchvision.transforms import transforms, v2
from scripts.score import jaccard_index

<a id="configuration"></a>
## ⚙️ | Configuration

In [7]:
PROJ_DIRECTORY = Path.cwd().parent
DATA_DIRECTORY = PROJ_DIRECTORY / "data"
DEV_DIRECTORY = PROJ_DIRECTORY / "data_dev"
IMAGES_DIRECTORY = DEV_DIRECTORY / "images"

<a id="datasets"></a>
## 🍚 | Datasets

In [8]:
train_meta = pd.read_csv(DEV_DIRECTORY / "train_metadata.csv", index_col="image_id")
train_labels = pd.read_csv(DEV_DIRECTORY / "train_labels.csv", index_col="image_id")

In [9]:
train_meta.shape, train_labels.shape

((25801, 2), (25801, 4))

In [10]:
train_labels.head()

,xmin,ymin,xmax,ymax
image_id,,,,
0001954c9f4a58f7ac05358b3cda8d20,0,277,345,709
00054819240f9d46378288b215dbcd3a,753,602,932,725
000dbf763348037b46558bbcb6a032ac,160,434,203,481
000e79208bebd8e84ce6c22fd8612a0d,70,534,211,586
000f13aff94499d03e3997afc55b0aa0,103,0,312,193


In [11]:
train_meta.head()

,spacecraft_id,background_id
image_id,,
0001954c9f4a58f7ac05358b3cda8d20,24,247
00054819240f9d46378288b215dbcd3a,14,10
000dbf763348037b46558bbcb6a032ac,19,17
000e79208bebd8e84ce6c22fd8612a0d,14,15
000f13aff94499d03e3997afc55b0aa0,28,15


In [12]:
# we didn't download the full training set, so add a column indicating which images are saved locally
train_meta["exists"] = train_meta.index.to_series().map(lambda x: (IMAGES_DIRECTORY / f"{x}.png").exists())
train_labels["exists"] = train_labels.index.to_series().map(lambda x: (IMAGES_DIRECTORY / f"{x}.png").exists())

# filter our metadata and training data down to only the images we have locally
train_meta = train_meta[train_meta.exists]
train_labels = train_labels[train_labels.exists]

print(train_meta.shape, train_labels.shape)

# Check indexes are the same
if train_meta.index.equals(train_labels.index):
    print("The indexes are the same.")
else:
    print("The indexes are different.")

(1579, 3) (1579, 5)
The indexes are the same.


In [13]:
# Examine an example image
example_img_idx = '000e79208bebd8e84ce6c22fd8612a0d'
example_img_path = Path(IMAGES_DIRECTORY, example_img_idx + '.png')
example_img = Image.open(str(example_img_path))
print(example_img.format, example_img.size, example_img.mode)

transform = transforms.Compose([
    transforms.ToTensor(),
])

example_img = transform(example_img)

if example_img.min() >= 0 and example_img.max() <= 1:
    print("Image pixel values are in the range [0, 1].")
else:
    print("Image pixel values are not in the range [0, 1].")

PNG (1280, 1024) RGB
Image pixel values are in the range [0, 1].


In [14]:
# Create pytorch DataSet
class SpacecraftDataset(torch.utils.data.Dataset):
    def __init__(self, labels_df, meta_df, imgs_dir=IMAGES_DIRECTORY, transforms=None):
        self.imgs_dir = imgs_dir
        self.labels_df = labels_df # dataframe of indexes and bbox coordinates
        self.meta_df = meta_df
        self.img_idxs = labels_df.index.tolist()
        self.transforms = transforms
    
    def __getitem__(self, idx):
        # Get image id and path
        img_id = self.img_idxs[idx]
        img_path = str(Path(self.imgs_dir, img_id + '.png'))
        n_objs = 1
        
        # Load image
        img = Image.open(img_path)

        # Get spacecraft id
        labels = torch.ones(n_objs, dtype=torch.int64)

        # Get bbox coordinates
        bbox = self._get_bbox(img_id)
        
        # Convert data to format needed by model.
        target = {}
        target['boxes'] = torch.from_numpy(bbox).reshape((1,4))
        target['labels'] = labels

        if self.transforms is not None:
            img = self.transforms(img)

        return img, target
    
    def __len__(self):
        return len(self.img_idxs)

    def _get_bbox(self, image_id):
        return self.labels_df.loc[image_id].loc[["xmin", "ymin", "xmax", "ymax"]].values.astype('int')

In [15]:
def collate_fn(batch):
    return list(zip(*batch))

In [16]:
# Create super small train and validation sets for quick iteration
dataset = SpacecraftDataset(train_labels, train_meta, transforms=transform)
train_ds = Subset(dataset, torch.arange(10))
valid_ds = Subset(dataset, torch.arange(10, 20))

train_dl = DataLoader(
    train_ds,
    batch_size=4,
    shuffle=False,
    collate_fn=collate_fn
)

valid_dl = DataLoader(
    valid_ds,
    batch_size=4,
    shuffle=False,
    collate_fn=collate_fn
)

## 🩼 | Helper Functions

In [17]:
# For evaluation
def images2tensors(img_ids):
    tensors = []
    for id in img_ids:
        img_path = str(Path(IMAGES_DIRECTORY, id + '.png'))
        img = read_image(img_path) / 255
        tensors.append(img)
    return tensors

In [18]:
def get_bbox(image_id, labels):
    """Get bbox coordinates as list from dataframe for given image id."""
    return labels.loc[image_id].loc[["xmin", "ymin", "xmax", "ymax"]].values

def display_image(image_id, images_dir=IMAGES_DIRECTORY, show_bbox=False, labels=None):
    """Display image given image ID. Annotate with bounding box if `show_bbox` is True."""
    img = cv2.imread(str(images_dir / f"{image_id}.png"))
    fig, ax = plt.subplots()
    # cv2 reads images as BGR order; we should flip them to RGB for matplotlib
    # ref: https://stackoverflow.com/questions/54959387/rgb-image-display-in-matplotlib-plt-imshow-returns-a-blue-image
    ax.imshow(np.flip(img, axis=-1))

    if show_bbox:
        xmin, ymin, xmax, ymax = get_bbox(image_id, labels)
        patch = Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, fill=False, edgecolor='white', linewidth=1)
        ax.add_patch(patch)

<a id="model"></a>
## 🤖 | Model

In [37]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT", trainable_backbone_layers=0)
# Adjust the model for spacecraft recognition task.
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

num_classes = 2 # 2 classes are spacecraft or background
in_features = model.roi_heads.box_predictor.cls_score.in_features

# replace pre-trained head with new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [38]:
model

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [39]:
# Test a forward pass
dataset = SpacecraftDataset(train_labels, train_meta, transforms=transform)
data_loader = DataLoader(
    dataset,
    batch_size=3,
    shuffle=True,
    collate_fn=collate_fn
)

# For training
images, targets = next(iter(data_loader))
images = list(images)
targets = list(targets)
output = model(images, targets)
print(output)

{'loss_classifier': tensor(0.5427, grad_fn=<NllLossBackward0>), 'loss_box_reg': tensor(0.1170, grad_fn=<DivBackward0>), 'loss_objectness': tensor(0.0112, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), 'loss_rpn_box_reg': tensor(0.0032, grad_fn=<DivBackward0>)}


In [40]:
# For inference
images, targets = next(iter(data_loader))
images = list(images)
model.eval()
predictions = model(images)
print(predictions)

[{'boxes': tensor([[ 441.8944,  373.9576,  513.3541,  456.3423],
        [ 535.3305,  397.9290,  555.3203,  411.7376],
        [ 446.8430,   57.5655,  716.2370,  566.0565],
        [ 436.5794,  358.6267,  563.2261,  473.6845],
        [  50.1064,  346.7328,  647.6163,  544.5023],
        [ 536.6421,  402.5836,  551.2484,  416.1528],
        [ 350.9720,  365.0808,  599.7423,  460.4680],
        [ 435.6753,  269.8577,  655.2936,  766.1903],
        [ 370.1882,   61.0875,  603.7681,  538.0585],
        [ 452.6169,  370.7442,  546.9471,  429.7160],
        [  42.2448,  161.0518,  735.3031,  516.0822],
        [ 441.1335,  407.1281,  511.3011,  489.5520],
        [ 299.0921,  237.9921,  989.8726,  456.3756],
        [ 422.5142,  349.8027,  520.3330,  525.1934],
        [ 541.5623,  395.2708,  566.3715,  412.2913],
        [ 320.7465,  103.9873,  936.9109,  579.4635],
        [ 338.6516,  346.1306,  979.3217,  571.0314],
        [ 372.0889,  380.6817,  523.2554,  450.2360],
        [ 446.105

<a id="loss-and-optimizer"></a>
## 🔍 | Loss & Optimizer

In [41]:
targets

({'boxes': tensor([[396, 247, 569, 515]]), 'labels': tensor([1])},
 {'boxes': tensor([[1000,    0, 1051,  146]]), 'labels': tensor([1])},
 {'boxes': tensor([[212, 119, 255, 333]]), 'labels': tensor([1])})

In [42]:
len(targets)

3

In [43]:
target_bboxes = []
for target in targets:
    target_bboxes.append(target['boxes'])
torch.stack(target_bboxes).shape


torch.Size([3, 1, 4])

In [44]:
target_bs = torch.stack([target['boxes'] for target in targets]).squeeze().numpy()
target_bs.shape

(3, 4)

In [55]:
predictions

[{'boxes': tensor([[ 441.8944,  373.9576,  513.3541,  456.3423],
          [ 535.3305,  397.9290,  555.3203,  411.7376],
          [ 446.8430,   57.5655,  716.2370,  566.0565],
          [ 436.5794,  358.6267,  563.2261,  473.6845],
          [  50.1064,  346.7328,  647.6163,  544.5023],
          [ 536.6421,  402.5836,  551.2484,  416.1528],
          [ 350.9720,  365.0808,  599.7423,  460.4680],
          [ 435.6753,  269.8577,  655.2936,  766.1903],
          [ 370.1882,   61.0875,  603.7681,  538.0585],
          [ 452.6169,  370.7442,  546.9471,  429.7160],
          [  42.2448,  161.0518,  735.3031,  516.0822],
          [ 441.1335,  407.1281,  511.3011,  489.5520],
          [ 299.0921,  237.9921,  989.8726,  456.3756],
          [ 422.5142,  349.8027,  520.3330,  525.1934],
          [ 541.5623,  395.2708,  566.3715,  412.2913],
          [ 320.7465,  103.9873,  936.9109,  579.4635],
          [ 338.6516,  346.1306,  979.3217,  571.0314],
          [ 372.0889,  380.6817,  523.2

In [54]:
predictions['boxes'][0]

TypeError: list indices must be integers or slices, not str

In [53]:
predicted_bs = torch.stack([pred['boxes'][0] for pred in predictions]).detach().numpy()
predicted_bs

array([[ 441.89438,  373.95758,  513.35406,  456.34225],
       [ 999.862  ,   43.084  , 1025.3412 ,   74.77467],
       [ 207.79663,  206.67293,  248.80527,  331.1286 ]], dtype=float32)

In [47]:
jaccard_index(predicted=predicted_bs, actual=target_bs)

array([0.12697746, 0.10779047, 0.47100891])

In [48]:
# Define loss and optimizer
def get_predicted_bboxes(preds):
    return torch.stack([pred['boxes'][0] for pred in preds])

def get_target_bboxes(targets):
    return torch.stack([target['boxes'] for target in targets])

def loss_fn(preds, targets):
    criterion = nn.SmoothL1Loss(reduction='mean')
    predicted_bboxes = get_predicted_bboxes(preds)
    target_bboxes = get_target_bboxes(targets)
    return criterion(predicted_bboxes, target_bboxes)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

<a id="training"></a>
## 🚂 | Training

In [49]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device

device(type='mps')

In [56]:
n_epochs = 2
# Define function to train the model
def train(model, n_epochs, train_dl, valid_dl):
    loss_hist_train = [0] * n_epochs
    jac_hist_train = [0] * n_epochs
    loss_hist_valid = [0] * n_epochs
    jac_hist_valid = [0] * n_epochs
    # model.to(device)
    for epoch in range(n_epochs):

        model.train() # Sets mode for dropout layer
        for images, targets in train_dl:
            # images = images.to(device)
            # targets = targets.to(device)
            # with torch.device(device):
            preds = model(images, targets)
            print(preds)
            loss = loss_fn(preds, targets)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            predicted_bboxes = get_predicted_bboxes(preds).detach().numpy()
            target_bboxes = get_predicted_bboxes(targets).numpy()
            loss_hist_train[epoch] += loss.item()*len(targets)
            jac_hist_train += jaccard_index(predicted_bboxes, target_bboxes)
        loss_hist_train[epoch] /= len(train_dl.dataset)
        jac_hist_train[epoch]  /= len(train_dl.dataset)

        model.eval() # Sets mode for dropout layer
        with torch.no_grad():
            for images, targets in valid_dl:
            #         images = images.to(device)
            #         targets = targets.to(device)
            #         with torch.device(device):
                preds = model(images, targets)
                loss = loss_fn(preds, targets)
                predicted_bboxes = get_predicted_bboxes(preds)
                target_bboxes = get_predicted_bboxes(targets)
                loss_hist_valid[epoch] += loss.item()*len(targets)
                jac_hist_valid += jaccard_index(predicted_bboxes, target_bboxes)
            loss_hist_train[epoch] /= len(valid_dl.dataset)
            jac_hist_train[epoch]  /= len(valid_dl.dataset)

        print(f'Epoch {epoch + 1} accuracy: '
              f'{jac_hist_train[epoch]:.4f} val_accuary '
              f'{jac_hist_valid[epoch]:.4f}')
        
    return loss_hist_train, loss_hist_valid, jac_hist_train, jac_hist_valid          
        

In [57]:
train(model, n_epochs, train_dl, valid_dl)

{'loss_classifier': tensor(0.5385, grad_fn=<NllLossBackward0>), 'loss_box_reg': tensor(0.1653, grad_fn=<DivBackward0>), 'loss_objectness': tensor(0.0090, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), 'loss_rpn_box_reg': tensor(0.0019, grad_fn=<DivBackward0>)}


TypeError: string indices must be integers

## 📋 | Result

The `train_labels.csv` contains the bounding box information for the target spacecraft in each image.

In [6]:
train_labels.head()

,xmin,ymin,xmax,ymax
image_id,,,,
0001954c9f4a58f7ac05358b3cda8d20,0,277,345,709
00054819240f9d46378288b215dbcd3a,753,602,932,725
000dbf763348037b46558bbcb6a032ac,160,434,203,481
000e79208bebd8e84ce6c22fd8612a0d,70,534,211,586
000f13aff94499d03e3997afc55b0aa0,103,0,312,193


Let's look at a few example images to get a feel for what's in this dataset.

### Review the benchmark submission scripts

Now let's take a look at the 2 files in `/example_benchmark`. This is the directory we are going to convert into a `submission.zip` file, which you can submit for the challenge.
* The `main.sh` shell script is a _required_ file for any submission to this challenge. Our code execution platform will run this script, and you can have it call other scripts and resources as needed for your submission.
* The `main.py` Python script is called by `main.sh` in this benchmark example, and this is where the work of generating predictions actually happens. There is no requirement that you also use Python here, but that's the approach we've taken since it is such a common one. The `main.py` script will iterate through all the images in the `submission_format.csv` and generate predictions using YOLO. If YOLO doesn't return a prediction, we simply generate a bounding box for the center of the image.

The `example_benchmark` directory should now contain the following files.  Note that we do need to include the `yolov8n.pt` file because the submission will have no internet access when running on our platform.
```
spacecraft-pose-object-detection-runtime/
└── example_benchmark/
    ├── main.py
    ├── main.sh
    └── yolov8n.pt
```

### Three commands to prepare your submission
To run and test the benchmark example, you just need to execute the following 3 commands:

1. [`make pull`](#make-pull)
2. [`make pack-benchmark`](#make-pack-benchmark)
3. [`make test-submission`](#make-test-submission)

These are defined in the project `Makefile` [here](https://github.com/drivendataorg/spacecraft-pose-object-detection-runtime/blob/main/Makefile). We'll walk through what each one does now.

<a id="make-pull"></a>

#### **`make pull`**

To ensure that all participants are using the same runtime environment, we have a publicly accessible docker image hosted on [Azure Container Registry](https://azure.microsoft.com/en-us/services/container-registry/).

The `make pull` command pulls the official version of the docker image and stores it locally. Having a local version of the competition image allows you to test your submission using the same image that is used during code execution.

> **Note:** This command can take a little while to run the first time you pull the image. But after that it will be relatively quick since you'll have all the layers cached locally. You don't need to pull the image again each time you test your submission, unless the image has changed.

In [17]:
!cd {PROJ_DIRECTORY} && make pull

docker pull spacecraftpose.azurecr.io/spacecraft-pose-object-detection:latest
latest: Pulling from spacecraft-pose-object-detection
Digest: sha256:025731e47e851c8f57363fc4d5a2e3fd44ec1a64f2089ba9d728ad7c8057f3a7
Status: Image is up to date for spacecraftpose.azurecr.io/spacecraft-pose-object-detection:latest
spacecraftpose.azurecr.io/spacecraft-pose-object-detection:latest


You should now have a local copy of the docker image, which you can verify by running:

In [27]:
!docker images | grep spacecraft-pose-object-detection

spacecraftpose.azurecr.io/spacecraft-pose-object-detection   latest    bc1d6eb59ba6   4 days ago     5.74GB


<a id="make-pack-benchmark"></a>

#### **`make pack-benchmark`** 
This command simply goes to your `example_benchmark` directory, zips the contents, and writes the zip archive to `submission/submission.zip`.

In [28]:
!cd {PROJ_DIRECTORY} && make pack-benchmark

mkdir -p submission/
cd example_benchmark; zip -r ../submission/submission.zip ./*
  adding: main.py (deflated 61%)
  adding: main.sh (deflated 26%)
  adding: yolov8n.pt (deflated 9%)


> **Note:** The `make pack-benchmark` command will check to see if you already have a `submission/submission.zip` and error if you do, so as not to overwrite existing work. If you already have this file, you'll need to manually remove it before running the command.

After running the above command, we should now have a new **`submission/submission.zip`**.
```
spacecraft-pose-object-detection-runtime/
├── benchmark_src/
│   ├── main.py
│   ├── main.sh
│   └── yolov8n.pt
└── submission/
    └── submission.zip   <---- new file, this is what gets submitted on platform
```

This is the file that we will eventually upload to the competition platform for code execution. But before doing that, we want to test it locally.

<a id="make-test-submission"></a>

#### **`make test-submission`** 
This command simulates what happens during actual code execution, launching an instance of the official Docker image and running the same inference process that runs on the competition platform. The required host directories are mounted on the container, and the entrypoint script `main.sh` is executed. Note that when testing locally the contents of your local `data/` directory will be mounted on the container, whereas when your submission is running on our platform, the unseen test set will be mounted as `data/`.

In [29]:
!cd {PROJ_DIRECTORY} && make test-submission

mkdir -p submission/
chmod -R 0777 submission/
docker run \
		-it \
		--network none \
		--mount type=bind,source="/Users/jsh/code/drivendata/spacecraft-pose-object-detection-runtime"/data,target=/code_execution/data,readonly \
		--mount type=bind,source="/Users/jsh/code/drivendata/spacecraft-pose-object-detection-runtime"/submission,target=/code_execution/submission \
		--shm-size 8g \
		--name spacecraft-pose-object-detection \
		--rm \
		bc1d6eb59ba6
+ main
+ tee /code_execution/submission/log.txt
+ cd /code_execution
+ curl --silent --connect-timeout 10 --max-time 12 www.example.com
+ data_directory=/code_execution/data
+ format_filename=/code_execution/data/submission_format.csv
+ (( i=0 ))
+ (( i<=5 ))
+ t=0
+ '[' -f /code_execution/data/submission_format.csv ']'
+ echo 'found /code_execution/data/submission_format.csv after 0 seconds; data is mounted'
found /code_execution/data/submission_format.csv after 0 seconds; data is mounted
+ break
+ '[' '!' -f /code_execution/data/submi

Once the test run has completed, we should now have a new file with our predictions at **`submission/submission.csv`**.
```
spacecraft-pose-object-detection-runtime/
├── benchmark_src/
│   ├── main.py
│   ├── main.sh
│   └── yolov8n.pt
└── submission/
    ├── submission.zip   <---- this is what gets submitted on platform
    └── submission.csv   <---- new file, predictions on test set
```
We also provide a scoring script that computes your score using the same calculation that's used for the public leaderboard. You can generate a score for your local testing with a command like the one below. Remember that this score will be computed on your local test set, and your score on the public leaderboard will be based on an unseen test set.
```
python scripts/score.py submission/submission.csv data/test_labels.csv
```

### Submitting to the platform
We're almost done. Assuming that our test run completed and the `submission.csv` looks correct, it's time to submit the code on the platform.

* Go to the [competition submissions page](https://www.drivendata.org/competitions/260/spacecraft-detection/submissions/) and upload your `submission/submission.zip`.
* Please be patient while your submission is uploaded and executed. Your job may be queued if other jobs are still pending.
* You can track the status of your submission on the [Code Execution Status](https://www.drivendata.org/competitions/260/submissions/code/) page. Logs will become available once the submission begins processing. To see them click on "View Log".

Once your submission has been successfully uploaded, you will see something like this on the [Code Execution Status](https://www.drivendata.org/competitions/260/submissions/code/) page:

![code execution status](https://drivendata-public-assets.s3.amazonaws.com/spacecraft-benchmark-code-status.jpg)

Please be patient while your code is running. You may want to follow the links to check the logs for your job, which are live updated as your code job progresses.

Once your job has completed, head over to the [Submissions](https://www.drivendata.org/competitions/260/spacecraft-detection/submissions/) page where you should be able to see your score. It will look something like this, except that we're sure you can do better than the benchmark!

![score](https://drivendata-public-assets.s3.amazonaws.com/spacecraft-benchmark-score.jpg)

**That's it! You're on your way to creating your own code submission!**

**Head over to the [competition](https://www.drivendata.org/competitions/260/spacecraft-detection/) homepage to get started. And have fun! We can't wait to see what you build!**

_Images courtesy of NASA._